<a href="https://colab.research.google.com/github/ophirmil12/Cyber-Project/blob/main/C_Course_2024_Bonus_2_Ophir_Miller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir=rtl>

#התקנות ושונות

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-tzyrtzxt
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-tzyrtzxt
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 326b0a57a80c6d0b4bad25ca7adf8138419ef1cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-8kgzblvb/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp7ln5nqys".


# check first cuda code

In [ ]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1



#run cuda code
<div dir=rtl>

*המטרה - להכפיל מטריצות כמות מסויימת של פעמים, ולהדפיס את זמן הריצה הכולל*

In [ ]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define SIZE 1000
#define BLOCK_SIZE 16

__global__ void matrix_multiply(float *A, float *B, float *C) {
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int bx = blockIdx.x;
    int by = blockIdx.y;

    int n = SIZE;
    float Csub = 0;

    for (int k = 0; k < n; k++) {
        float Ak = A[bx * n * BLOCK_SIZE + tx * n + k];
        float Bk = B[k * n * BLOCK_SIZE + by * n + ty];
        Csub += Ak * Bk;
    }

    __syncthreads();

    if (tx == 0 && ty == 0) {
        C[bx * n * BLOCK_SIZE + by * n + ty * n + tx] = Csub;
    }
}

int main() {
    for (int times = 0; times < 10; times++)
      {
      float *A, *B, *C;
      A = (float *)malloc(SIZE * SIZE * sizeof(float));
      B = (float *)malloc(SIZE * SIZE * sizeof(float));
      C = (float *)malloc(SIZE * SIZE * sizeof(float));

      // Initialize matrix A with random numbers
      for (int i = 0; i < SIZE * SIZE; i++) {
          A[i] = (float)rand() / RAND_MAX;
      }

      // Copy matrix A to matrix B
      for (int i = 0; i < SIZE * SIZE; i++) {
          B[i] = A[i];
      }

      float *d_A, *d_B, *d_C;
      cudaMalloc((void **)&d_A, SIZE * SIZE * sizeof(float));
      cudaMalloc((void **)&d_B, SIZE * SIZE * sizeof(float));
      cudaMalloc((void **)&d_C, SIZE * SIZE * sizeof(float));

      cudaMemcpy(d_A, A, SIZE * SIZE * sizeof(float), cudaMemcpyHostToDevice);
      cudaMemcpy(d_B, B, SIZE * SIZE * sizeof(float), cudaMemcpyHostToDevice);

      dim3 threads(BLOCK_SIZE, BLOCK_SIZE);
      dim3 blocks((SIZE + BLOCK_SIZE - 1) / BLOCK_SIZE, (SIZE + BLOCK_SIZE - 1) / BLOCK_SIZE);

      clock_t start = clock();

      // Multiply matrix A by itself 100 times on the GPU
      for (int i = 0; i < 100; i++) {
          matrix_multiply<<<blocks, threads>>>(d_A, d_B, d_C);
          cudaMemcpy(A, d_A, SIZE * SIZE * sizeof(float), cudaMemcpyDeviceToHost);
          cudaMemcpy(B, d_B, SIZE * SIZE * sizeof(float), cudaMemcpyDeviceToHost);
          cudaMemcpy(C, d_C, SIZE * SIZE * sizeof(float), cudaMemcpyDeviceToHost);
      }

      clock_t end = clock();

      printf("Execution time: %f seconds\n", (double)(end - start) / CLOCKS_PER_SEC);

      cudaFree(d_A);
      cudaFree(d_B);
      cudaFree(d_C);

      free(A);
      free(B);
      free(C);
    }
}

Execution time: 0.056758 seconds
Execution time: 0.000046 seconds
Execution time: 0.000047 seconds
Execution time: 0.000049 seconds
Execution time: 0.000047 seconds
Execution time: 0.000049 seconds
Execution time: 0.000052 seconds
Execution time: 0.000046 seconds
Execution time: 0.000050 seconds
Execution time: 0.000043 seconds



# run C code
<div dir=rtl>

*אותה מטרה, הפעם בשפת C*
* ראה פירוט בהמשך.

In [ ]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define SIZE 1000

int main() {
    for (int times = 0; times < 10; times++)
      {
      int i, j, k;
      double **A, **B, **C;
      clock_t start, end;
      double time_taken;

      // Allocate memory for matrices
      A = (double **)malloc(SIZE * sizeof(double *));
      B = (double **)malloc(SIZE * sizeof(double *));
      C = (double **)malloc(SIZE * sizeof(double *));
      for (i = 0; i < SIZE; i++) {
          A[i] = (double *)malloc(SIZE * sizeof(double));
          B[i] = (double *)malloc(SIZE * sizeof(double));
          C[i] = (double *)malloc(SIZE * sizeof(double));
      }

      // Initialize matrices with random numbers
      srand(time(NULL));
      for (i = 0; i < SIZE; i++) {
          for (j = 0; j < SIZE; j++) {
              A[i][j] = (double)rand() / RAND_MAX;
              B[i][j] = A[i][j];
              C[i][j] = 0.0;
          }
      }

      // Start timing
      start = clock();

      // Multiply matrix A by itself 100 times
      for (k = 0; k < 100; k++) {
          for (i = 0; i < SIZE; i++) {
              for (j = 0; j < SIZE; j++) {
                  for (int l = 0; l < SIZE; l++) {
                      C[i][j] += A[i][l] * B[l][j];
                  }
              }
          }
          // Copy matrix C to matrix A
          for (i = 0; i < SIZE; i++) {
              for (j = 0; j < SIZE; j++) {
                  A[i][j] = C[i][j];
              }
          }
      }

      // Stop timing
      end = clock();

      // Calculate time taken
      time_taken = ((double)(end - start)) / CLOCKS_PER_SEC;

      // Print time taken
      printf("Time taken to multiply a %d x %d matrix by itself 100 times: %f seconds\n", SIZE, SIZE, time_taken);

      // Free memory
      for (i = 0; i < SIZE; i++) {
          free(A[i]);
          free(B[i]);
          free(C[i]);
      }
      free(A);
      free(B);
      free(C);

      return 0;
      }
}

Time taken to multiply a 1000 x 1000 matrix by itself 100 times: 1014.913764 seconds



<div dir=rtl>
# בעיה קטנה... קודה כל כך יותר מהיר מC

* ההרצה של מטריצות בגודל 1000 הייתה ארוכה מידי...
* נריץ על מטריצות בגודל 100, ונבחין כי בכל זאת זמן ההרצה היה יותר מפי 10^4, וואו!
* עצרתי את הריצה לאחר זמן ממושך, התוכנית הספיקה לעשות איטרציה יחידה, וקיבלנו שלקח כ-1000 שניות להריץ.


In [ ]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define SIZE 100

int main() {
    for (int times = 0; times < 10; times++)
      {
      int i, j, k;
      double **A, **B, **C;
      clock_t start, end;
      double time_taken;

      // Allocate memory for matrices
      A = (double **)malloc(SIZE * sizeof(double *));
      B = (double **)malloc(SIZE * sizeof(double *));
      C = (double **)malloc(SIZE * sizeof(double *));
      for (i = 0; i < SIZE; i++) {
          A[i] = (double *)malloc(SIZE * sizeof(double));
          B[i] = (double *)malloc(SIZE * sizeof(double));
          C[i] = (double *)malloc(SIZE * sizeof(double));
      }

      // Initialize matrices with random numbers
      srand(time(NULL));
      for (i = 0; i < SIZE; i++) {
          for (j = 0; j < SIZE; j++) {
              A[i][j] = (double)rand() / RAND_MAX;
              B[i][j] = A[i][j];
              C[i][j] = 0.0;
          }
      }

      // Start timing
      start = clock();

      // Multiply matrix A by itself 100 times
      for (k = 0; k < 100; k++) {
          for (i = 0; i < SIZE; i++) {
              for (j = 0; j < SIZE; j++) {
                  for (int l = 0; l < SIZE; l++) {
                      C[i][j] += A[i][l] * B[l][j];
                  }
              }
          }
          // Copy matrix C to matrix A
          for (i = 0; i < SIZE; i++) {
              for (j = 0; j < SIZE; j++) {
                  A[i][j] = C[i][j];
              }
          }
      }

      // Stop timing
      end = clock();

      // Calculate time taken
      time_taken = ((double)(end - start)) / CLOCKS_PER_SEC;

      // Print time taken
      printf("Time taken to multiply a %d x %d matrix by itself 100 times: %f seconds\n", SIZE, SIZE, time_taken);

      // Free memory
      for (i = 0; i < SIZE; i++) {
          free(A[i]);
          free(B[i]);
          free(C[i]);
      }
      free(A);
      free(B);
      free(C);

      }
    return 0;
}

Time taken to multiply a 100 x 100 matrix by itself 100 times: 0.523293 seconds
Time taken to multiply a 100 x 100 matrix by itself 100 times: 0.545763 seconds
Time taken to multiply a 100 x 100 matrix by itself 100 times: 0.500590 seconds
Time taken to multiply a 100 x 100 matrix by itself 100 times: 0.477688 seconds
Time taken to multiply a 100 x 100 matrix by itself 100 times: 0.479112 seconds
Time taken to multiply a 100 x 100 matrix by itself 100 times: 0.551137 seconds
Time taken to multiply a 100 x 100 matrix by itself 100 times: 0.812585 seconds
Time taken to multiply a 100 x 100 matrix by itself 100 times: 0.816380 seconds
Time taken to multiply a 100 x 100 matrix by itself 100 times: 0.672102 seconds
Time taken to multiply a 100 x 100 matrix by itself 100 times: 0.474385 seconds



<div dir=rtl>

#חלק זה נכתב בעזרת מודל שפה
* כמו כן, הקודים נכתבו חלקית בעזרת מודל שפה (חדש!) מיועד לתכנות blackbox.ai (ממליץ בחום)


[ קישור לצ'אט עם המודל](https://www.blackbox.ai/share/f2896d92-39b4-4f89-a705-12ed15845464)



להלן כמה סיבות לכך ש-CUDA (GPUs עם CUDA) יכול להיות מהיר יותר מ-C (CPU) עבור סוגים מסוימים של חישובים:

*מקביליות*: CUDA מאפשר ביצוע של מספר שרשורים במקביל, מה שיכול להאיץ משמעותית את הביצוע של סוגים מסוימים של חישובים.

*היררכיית זיכרון*: ל-CUDA יש היררכיית זיכרון עמוקה, הכוללת אוגרים, זיכרון משותף וזיכרון גלובלי, שניתן להשתמש בהם כדי לייעל את ביצועי החישובים.

*פעולות וקטוריות*: CUDA תומך בפעולות וקטוריות, שניתן להשתמש בהן לביצוע מספר פעולות במקביל.

*ביצוע ליבה*: CUDA מאפשר ביצוע של ליבות, שהן פונקציות שמבוצעות על ה-GPU. זה מאפשר הורדה של חישובים מהמעבד ל-GPU, מה שיכול לשפר את הביצועים של המערכת הכוללת.
במקביל: CUDA מאפשר ביצוע במקביל של מספר גרעינים, מה שיכול לשפר עוד יותר את ביצועי המערכת.

*ספריות אופטימליות*: CUDA מספקת ספריות אופטימליות, כגון cuBLAS ו-cuFFT, שניתן להשתמש בהן לביצוע אלגברה ליניארית נפוצה ופעולות טרנספורמציה מהירה של פורייה.
האצת חומרה: GPUs התומכים ב-CUDA תוכננו במיוחד עבור מחשוב בעל ביצועים גבוהים, ולעתים קרובות יש להם יותר ליבות ורוחב פס זיכרון גבוה יותר מאשר למעבדים. זה יכול לגרום לביצועים מהירים יותר עבור סוגים מסוימים של חישובים.

חשוב לציין ש-CUDA לא תמיד מהיר יותר מ-C עבור כל סוגי החישובים. הביצועים של CUDA ו-C יכולים להיות תלויים בבעיה הספציפית הנפתרת, בגודל הנתונים ובחומרה שבה נעשה שימוש. במקרים מסוימים, CUDA עשוי להיות מהיר משמעותית מ-C, בעוד שבמקרים אחרים, ההבדל בביצועים עשוי להיות זניח.


בנוסף, תכנות CUDA דורש מערך מיומנויות שונה מתכנות C, וזה יכול להיות מורכב יותר לכתוב וניפוי באגים בקוד CUDA. חשוב לשקול היטב את ההחלפות בין CUDA ל-C כאשר מחליטים באילו להשתמש עבור פרויקט מסוים.




**בכל אופן,**

אני באופן אישי מעדיף לעבוד עם שפות ביניים. כמסקנה מהפרוייקט הזה, והאחד שלפניו (ההשוואה בין פייתון ל-C), אני מעריך שהשימוש בשפות ביניים אידיאלי.
שפות נמוכות כיום נחשבות למסורבלות מאוד, ובמקרים רבים לא שמישות. שפות גבוהות (כמו פייתון) לוקחות המון זמן ריצה - אך עוזרות בתחילת הלימוד לקליטה של קונספטים מתקדמים במדעי המחשב.

</div>